In [1]:
!pip install gradio opencv-python torch torchvision -q

In [2]:
import cv2
import numpy as np
import gradio as gr
from PIL import Image
import requests, os

In [8]:
# ---------------------------
# 1. Dummy Era Classifier (replace with trained model)
# ---------------------------
class DummyEraClassifier:
    eras = ["1900s", "1930s", "1950s", "1980s"]

    def predict(self, img):
        # For demo: pick randomly (replace with CNN)
        return np.random.choice(self.eras)

era_model = DummyEraClassifier()

In [9]:
# ---------------------------
# 2. Simple "Base Colorization" (convert grayscale -> fake color)
#    (Replace with DeOldify or Zhang model for real results)
# ---------------------------
def simple_colorize(gray_img):
    gray = np.array(gray_img.convert("L")) / 255.0
    lab = np.zeros((gray.shape[0], gray.shape[1], 3))
    lab[:, :, 0] = gray * 100  # L channel
    lab[:, :, 1:] = np.random.uniform(-20, 20, size=(gray.shape[0], gray.shape[1], 2))
    return Image.fromarray((color.lab2rgb(lab) * 255).astype("uint8"))


In [10]:
# ---------------------------
# 3. Apply Palette Adjustment per Era
# ---------------------------
era_palettes = {
    "1900s": np.array([180, 150, 120]),   # sepia
    "1930s": np.array([200, 180, 160]),
    "1950s": np.array([220, 200, 200]),   # pastel
    "1980s": np.array([250, 180, 220])    # neon-ish
}

def apply_palette(img, era):
    arr = np.array(img).astype(np.float32)
    tint = era_palettes[era]
    arr = 0.7 * arr + 0.3 * tint  # blend
    return Image.fromarray(np.clip(arr, 0, 255).astype("uint8"))

In [11]:
# ---------------------------
# 4. Full Pipeline
# ---------------------------
def process(img, manual_era=None):
    auto_era = era_model.predict(img)
    era = manual_era if manual_era != "Auto" else auto_era

    base_col = simple_colorize(img)
    final_col = apply_palette(base_col, era)

    return final_col, f"Predicted Era: {auto_era} | Using: {era}"


In [18]:
import gradio as gr
import numpy as np
import cv2
from PIL import Image, ImageEnhance

# Apply fake colorization + timeline filter
def time_based_colorize(image, era):
    img = np.array(image.convert("RGB"))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # Base fake colorization
    img[:, :, 0] = img[:, :, 0] * 0.6   # reduce blue
    img[:, :, 1] = img[:, :, 1] * 0.9   # reduce green
    img[:, :, 2] = img[:, :, 2] * 1.4   # boost red
    img = np.clip(img, 0, 255).astype("uint8")
    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # Apply "era" style
    if era == "1900s":
        pil_img = pil_img.convert("L").convert("RGB")  # grayscale
        pil_img = ImageEnhance.Color(pil_img).enhance(0.3)  # muted
        pil_img = ImageEnhance.Brightness(pil_img).enhance(1.1)
    elif era == "1950s":
        pil_img = ImageEnhance.Color(pil_img).enhance(1.2)  # pastel-like
        pil_img = ImageEnhance.Brightness(pil_img).enhance(1.1)
        pil_img = ImageEnhance.Contrast(pil_img).enhance(0.9)
    elif era == "1980s":
        pil_img = ImageEnhance.Color(pil_img).enhance(1.6)  # vibrant
        pil_img = ImageEnhance.Brightness(pil_img).enhance(1.2)
        pil_img = ImageEnhance.Contrast(pil_img).enhance(1.2)

    return pil_img

# Gradio interface
demo = gr.Interface(
    fn=time_based_colorize,
    inputs=[
        gr.Image(type="pil", label="Upload Grayscale Image"),
        gr.Radio(["1900s", "1950s", "1980s"], label="Choose Era", value="1950s")
    ],
    outputs=gr.Image(type="pil", label="Time-based Colorized Image"),
    title="Time-Based Historical Colorization",
    description="Upload a black & white photo and colorize it according to the chosen time period."
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fd236bcccd2e864163.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
